In [18]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Problem Statement - Forecast the Price for Last Three Months based on the past 29 months Data.
#### Q. Using Simple Exponential Smoothing model - what's the optimised value of Smoothing Level when Algorithm is trained with Optimised = True. 
#### Options : 0.11, 0.22, 0.1, 0.80 

## Solution : It involves following Steps:
### 1. Data Preparation 
### 2. Training and Test Data Preparation
### 3. Training with SimpleExponential Smoothing Model 
### 4. ForeCasting the Trained Model for Next Three Months. 
### 5. Model Analysis with RMSE and MAPE.
### 6. Repeat with Douple Exponential Smoothing(Holt's Methid) and Triple Exponential Model Smoothing (Holtwinters' method)
**Let's begin with step by step.** 

## 1. Prepare Preparation

In [3]:
data = [
    [0, 217.4],
    [1, 255.7],
    [2, 244.3],
    [3, 236.1],
    [4, 228.7],
    [5, 262.9],
    [6, 284.5],
    [7, 231.4],
    [8, 236.5],
    [9, 316.0],
    [10, 376.9],
    [11, 429.0],
    [12, 365.5],
    [13, 439.2],
    [14, 416.0],
    [15, 446.6],
    [16, 530.7],
    [17, 674.7],
    [18, 623.7],
    [19, 576.2],
    [20, 611.1],
    [21, 704.1],
    [22, 739.0],
    [23, 966.6],
    [24, 966.2],
    [25, 1189.1],
    [26, 1081.7],
    [27, 1435.2],
    [28, 2191.8],
    [29, 2420.7],
    [30, 2856.0],
    [31, 4268.8]
]

# Convert data to DataFrame
df = pd.DataFrame(data, columns=["Months", "Price"])
df.head()

,Months,Price
0,0,217.4
1,1,255.7
2,2,244.3
3,3,236.1
4,4,228.7


In [4]:
df.describe()

,Months,Price
count,32.000000,32.000000
mean,15.500000,838.196875
std,9.380832,908.925885
min,0.000000,217.400000
25%,7.750000,279.100000
50%,15.500000,488.650000
75%,23.250000,966.300000
max,31.000000,4268.800000


## 2. Training and Test Data Preparation

In [5]:
train_data = df["Price"][:-3]
test_data = df["Price"][-3:]
print(f"My Training Data : {train_data} \n and My Test Data: {test_data}")

My Training Data : 0      217.4
1      255.7
2      244.3
3      236.1
4      228.7
5      262.9
6      284.5
7      231.4
8      236.5
9      316.0
10     376.9
11     429.0
12     365.5
13     439.2
14     416.0
15     446.6
16     530.7
17     674.7
18     623.7
19     576.2
20     611.1
21     704.1
22     739.0
23     966.6
24     966.2
25    1189.1
26    1081.7
27    1435.2
28    2191.8
Name: Price, dtype: float64 
 and My Test Data: 29    2420.7
30    2856.0
31    4268.8
Name: Price, dtype: float64


## 3. Training the model with Simple exponential

In [6]:
def train(train_data, optimized=True, trend=None, seasonal=None, seasonal_periods=None, smoothing_label=None):
    model = ExponentialSmoothing(train_data, trend=trend, seasonal=seasonal, seasonal_periods=seasonal_periods)
    model_fit = model.fit(optimized=optimized, smoothing_level=smoothing_label)
    return model_fit

## 4. ForeCast for len of Test Data using Model

In [7]:
def predict(test_data, model):
    forcasted = model.forecast(steps=len(test_data))
    return forcasted

## 5. Evaluate Model with RMSE and MAPE metrics

In [8]:
def evaluate(test_data, forcasted):
    mape = (np.abs(test_data - forcasted) / test_data).mean() * 100
    
    # Calculate Root Mean Squared Error (RMSE)
    rmse = np.sqrt(np.mean((test_data - forcasted)**2))

    print(f"Mean Absolute Percentage Error (MAPE): {mape:.3f}%")
    print(f"Root Mean Squared Error (RMSE): {rmse:.3f}")

In [17]:
model = train(train_data, smoothing_label=0.995)
print(model.params)
forcasted = predict(test_data, model)
print(forcasted)
evaluate(test_data, forcasted)

{'smoothing_level': 0.995, 'smoothing_trend': nan, 'smoothing_seasonal': nan, 'damping_trend': nan, 'initial_level': 217.59121386343764, 'initial_trend': nan, 'initial_seasons': array([], dtype=float64), 'use_boxcox': False, 'lamda': None, 'remove_bias': False}
29    2188.008176
30    2188.008176
31    2188.008176
dtype: float64
Mean Absolute Percentage Error (MAPE): 27.249%
Root Mean Squared Error (RMSE): 1268.865


**As Seen above MAPE is 27.25% and RMSE is 1268.86 - that means model wouldn't perform good. Lets try with variations of HOLT's model** 

## Repeat with Double Exponential -- Trend 

In [10]:
model = train(train_data, smoothing_label=None, trend="add")
forcasted = predict(test_data, model)
print(model.params)
print(forcasted)
evaluate(test_data, forcasted)

{'smoothing_level': 0.995, 'smoothing_trend': 0.4264285714285714, 'smoothing_seasonal': nan, 'damping_trend': nan, 'initial_level': 222.41999999999987, 'initial_trend': 5.260000000000016, 'initial_seasons': array([], dtype=float64), 'use_boxcox': False, 'lamda': None, 'remove_bias': False}
29    2607.261737
30    3025.674541
31    3444.087345
dtype: float64
Mean Absolute Percentage Error (MAPE): 10.989%
Root Mean Squared Error (RMSE): 497.911


**Double Exponential Smoothing Performs better than Simple Exponential Smoothing with Smoothing Label = 0.6**

## Repeat with Triple Exponential -- Trend & Seasonal

In [11]:
model = train(train_data,smoothing_label=None,  trend="add", seasonal="add", seasonal_periods=6)
forcasted = predict(test_data, model)
print(model.params)
print(forcasted)
evaluate(test_data, forcasted)

{'smoothing_level': 0.9953265659907837, 'smoothing_trend': 0.4969758853374483, 'smoothing_seasonal': 0.004465327903089963, 'damping_trend': nan, 'initial_level': 200.44162692873093, 'initial_trend': 18.31293002024134, 'initial_seasons': array([ 13.89519488,   2.41561662, -75.18374266, -55.5365589 ,
        27.03297925,  86.27775981]), 'use_boxcox': False, 'lamda': None, 'remove_bias': False}
29    2685.709930
30    3047.229447
31    3470.548693
dtype: float64
Mean Absolute Percentage Error (MAPE): 12.114%
Root Mean Squared Error (RMSE): 497.997


**As seen from MAPE & RMSE values ForeCastings are better with Triple Exponential Holt's method than SimpleExpomnential or Double Exponential Method**

## Repeat with Double Exponential - Trend & Seasonal with Trend being Multiplicative

In [12]:
model = train(train_data,smoothing_label=None,  trend="mul", seasonal="mul", seasonal_periods=6)
forcasted = predict(test_data, model)
print(model.params)
print(forcasted)
evaluate(test_data, forcasted)

{'smoothing_level': 0.5975736399801944, 'smoothing_trend': 0.5975733810152634, 'smoothing_seasonal': 8.3091359396345e-05, 'damping_trend': nan, 'initial_level': 203.83521636989593, 'initial_trend': 1.0268708768315418, 'initial_seasons': array([1.20173523, 1.22138542, 1.00144091, 1.08669546, 1.31542165,
       1.39321733]), 'use_boxcox': False, 'lamda': None, 'remove_bias': False}
29    2751.826671
30    2902.236862
31    3606.501843
dtype: float64
Mean Absolute Percentage Error (MAPE): 10.271%
Root Mean Squared Error (RMSE): 428.338


**As seen from MAPE & RMSE, Triple Exponential Smoonthing with Multicative for seasonal forecasts better than with Additive** 

In [13]:
model = train(train_data,smoothing_label=None,  trend="add", seasonal="mul", seasonal_periods=6)
forcasted = predict(test_data, model)
print(model.params)
print(forcasted)
evaluate(test_data, forcasted)

{'smoothing_level': 0.8889311096719034, 'smoothing_trend': 0.8889014238582772, 'smoothing_seasonal': 0.1110651640358054, 'damping_trend': nan, 'initial_level': 201.5390922813669, 'initial_trend': 18.026352675874282, 'initial_seasons': array([1.26374379, 1.14043606, 0.89493085, 1.07850237, 1.23962812,
       1.39070203]), 'use_boxcox': False, 'lamda': None, 'remove_bias': False}
29    3298.820609
30    3435.657516
31    3785.409291
dtype: float64
Mean Absolute Percentage Error (MAPE): 22.632%
Root Mean Squared Error (RMSE): 668.522


**As seen from MAPE & RMSE, Triple Exponential Smoonthing with Additive for Trend and Multicative for seasonal forecasts arenot better than trend and seasonal both being mulplictaive** 